# Fully Connected Neural Networks in TensorFlow's Keras

We will use dense neural networks in Keras to solve a simple regression problem. 

The dataset is a collection of simulated particle events from [Pythia](http://home.thep.lu.se/~torbjorn/Pythia.html). Specifically, you will construct a dense neural network which will learn how to calculate the invariant mass of a particle from its energy and momentum.

First, import `numpy`, `tensorflow`, and `pylab` and load the data.

In [ ]:
import numpy as np
import tensorflow as tf
import pylab as plt

# Prevent TensorFlow from showing us deprecation warnings
tf.logging.set_verbosity(tf.logging.ERROR)

# MPK comp hack. You should not need this.
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# load the data, which is stored as a numpy array data type
energy = np.load("homogenous-16-particle-events-energy.npy")

The individual particles must be extracted next. This data is currently stored by event, each of which contains 16 particles which each have an x-momentum, a y-momentum, a z-momentum, an energy, and a charge. Each event has these numbers strong together one after the other for each particle and thus has 80 numbers. Using numpy's reshape method we can seperate out all of the individual events. 

In [ ]:
#Here we rearrange the data within each of the events to isolate particles
energy=np.reshape(energy,(len(energy),16,5))

#Use another call of reshape to combine all events
#Complete me:

#Try printing the energy to make sure this worked
#print()

Now that we have the training data inputs, we need to calculate the target value. This is the invariant mass of the particle. This is an easy calculation to do and not something for which we really needneural networks, but it serves as a simple regression problem. We can do this calculation using the formula
$$m^2=E^2-||\textbf{p}||^2$$
where $m, E$, and $\textbf{p}$ are all in 

In [ ]:
#This will be much easier if you take the transpose of energy by calling energy.T.
#There will also be roundoff error, so consider that before taking the squareroot. 

#Complete me:


There are several hundred thousand datapoints in this dataset which may be a lot more a problem as simple as this. Create a test dataset with just 1000 examples.

In [ ]:
#Slicing allows you select a subset of an array.
#This can be done like this: smallerArray = largerArray[:100]

#Complete me:


Next, make a histogram of the target data to make sure that we are seeing masses of real particles. As this data has limited precision, this will not resolve electrons very well, but protons, pions, and massless particles should be clearly visible.

In [ ]:
plt.hist(target,bins=100)
plt.show()

Now it is time to actually train the first neural network. Start with a simple one hidden layer network with 5 neurons, and Relu activation. Train for 30 epochs with a batch size of 256, an Adam optimizer with a learning rate of 0.1, using mean squared error loss. Additionally, use a validation split of 0.8 when fitting to check for overtraining. Information on how to implement these features can be found here:
<https://www.tensorflow.org/api_docs/python/tf/keras>.
Check out Sequential underneath models and Dense under layers.


In [ ]:
model=tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(5,input_shape=(5,),activation="relu")) #Add the hidden layer

#Add the output layer yourself
#It should not have an activation function
#Complete me:

model.compile(tf.keras.optimizers.Adam(lr=0.1),loss=tf.keras.losses.MeanSquaredError()) #Adam optimizer and mean squared error loss
model.fit(smallEnergy,smallTarget,epochs=100, batch_size=256, validation_split=0.8)

In [ ]:
plt.plot(history["loss"])
plt.plot(history["val_loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")

The model is learning, but we can do better. Perhaps you are thinking that we did not have enough fitting paramters to accurately represent the mapping. Remedy this by increasing the number of hidden neurons to 20.

In [ ]:
#Use the same code as in the previous cell
#simply change the number of neurons in the hidden layer

#Complete me:


We see that we got little improvement here. Another hyperparameter to adjust is *batch size*, which is the number of training examples used to calculate the gradient at each iteration. While you may initially think that a higher batch size leads to faster or more accurate training, in practice this is not true. The "noise" that arises from using less training examples at each iteration can actually help find the global minimum of the loss function.
(See here for more info: https://arxiv.org/pdf/1609.04836.pdf)

Try decreasing the batch size to 16.

In [ ]:
#Complete me:


This is starting to do better but has significant room for improvemnt. 

Another hyperparameter to tune is the *learning rate*. 

 - If the learning rate is too high, we are taking too large of a step in the gradient descent at each iteration and will miss narrow minima in the loss function. 
 - If the learning rate is too small, then we are not traveling far enough in each iteration and we will take far too long to reach a minimum. 

Perhpas the learning rate is to high and the network can't fine tune. Try decreasing the learning rate to 0.001.

In [ ]:
#complete me:


This is not really that mutch better, but now there is evidence of *overtraining* or *overfitting* -- the training loss is so much lower than the validation loss. 

A common fix to this is adding *dropout layers*. Try adding a dropout layer with dropout rate of 0.5. <https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout>

In [ ]:
#Dropout layers are located under tf.keras.layers. 
#They take the dropout rate as their only argument.

#Complete me:


This clearly stopped the overtraining problem, but it still isn't training well. Now, try training on the full dataset with a more reasonable validation split of 0.2. Use the a single hidden layer with 20 neurons, a learning rate of 0.001, and a batch size of 256. Just run it for 10 epochs.

In [ ]:
#Complete me:


This clearly resulted in a significant improvement and shows how important having a large enough dataset it. Moving on to the choice in activation functions, ReLU is not the only available choice, although it is one of the most popular ones currently. Try training a network using a sigmoid or tanh activation function.

In [ ]:
#Simply change relu to sigmoid or tanh to change the activation function

#Complete me:


Next, try adding 2 new hidden layers to the network. Use the relu activation function.

In [ ]:
#Complete me:


Clearly, adding more layers helps improve the quality of the network. There is a limit to now effective this is though. Try having 5 hidden layers.

In [ ]:
#Complete me:


Now, see what happens when you increase the number of neurons per layer from 20 to 50 in the 3 hidden layer model. Consider how they perform compared to Relu now. 

In [ ]:
#Complete me:


Try using the sigmoid and the tanh activation functions again and compare them to relu.

In [ ]:
#Complete me:


This difference in performace, especially with the sigmoid function, is known as the vanishing gradient problem. If the value for any one the neurons gets too far away from 0, the gradient for sigmoid and tanh gets really close to 0. This means that for deeper networks it is much more difficult to update the first layers as their gradient is so small. Now, remove the fifth column from the input data, the charge, and see what happens when training. Why do you thing including charge has such a large impact?

In [ ]:
#Complete me:


Finally, there are other options for the loss function. Try experimenting with mean squared error. Other optimizers that can be used are sgd, rmsprop, adagrad, adadelta, adamax, and nadam. <https://www.tensorflow.org/api_docs/python/tf/keras/optimizers>

<https://www.tensorflow.org/api_docs/python/tf/keras/losses>